In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import requests
from urllib.parse import urlencode, quote_plus
import numpy as np
import sys
from dotenv import find_dotenv, load_dotenv
import os
import pandas as pd

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sayemkamal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sayemkamal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sayemkamal/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# **Citing this code**
This code is the second version of a Expertise finding tool developed by Volz et al. 2023 (https://ui.adsabs.harvard.edu/abs/2023AAS...24210207V/abstract).<br>
It utilizes NASA ADS API to query for articles (refereed or not) in the "Astronomy" database (cite ADS).
Please, cite "Helfenbein et al. 2023 (in prep) and refer to the README file in the github.

**Directory set up**<br>
The file *stopwords.txt* is utilized to create meaningful N-grams. Make sure to provide an accurate path in the following cell.<br> Also, the path will be used by the code in other instances to identify other useful files.

In [3]:
path_stop= '/Users/sayemkamal/NASA_Project/ReviewerExtractor/codeV3/'
stop_file='stopwords.txt'
stop_dir=path_stop+stop_file
sys.path.append(path_stop)



In [4]:
#token = 'Your own token from ADS API page ' #Insert your API token
token = 'WIhvOVWrt4ksJRFq7P0fuYToniASsS5Wlp5NC28V' 


In [5]:
#For the TextAnalysis File, please refer to M. Volze et al. 2023
import TextAnalysis as TA
import ADSsearcherpkg as AP

# **Example 1: Searching expertises of a single person based on their name**

The search will focus on papers published by a specific author in the past 15 years independently of the current affiliation:<br>
The format for a single author search is as follows: **"Last, First"**<br>
In the following example we search for Dr. Joshua Pepper expertise. <br>
**Note:** the user can decide to query ONLY refereed paper adding, before the token keyword the following keyword:<br>
**refereed="property:refereed"**


In [6]:
datf=AP.ads_search(name="Cucchiara, Antonino",
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
author:"^Cucchiara, Antonino"

I am now querying ADS.



In [7]:
import inspect
print(inspect.signature(AP.ads_search))

(name=None, institution=None, year=None, refereed='property:notrefereed OR property:refereed', token=None, stop_dir=None, second_auth=False)


In [8]:
# To display the data frame run the following:
datf
print(len(datf.iloc[0]['Title']))
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

139


# Testing Second Author

In [9]:
datf2=AP.ads_search(name="Cucchiara, Antonino",
               token=token, stop_dir=stop_dir, second_auth=True)

I will search for papers matching the following criteria:
pos(author:"^Cucchiara, Antonino",1) OR pos(author:"Cucchiara, Antonino",2)

I am now querying ADS.



In [10]:
(len(datf2.iloc[0]['Title']))

284

# **Example 2: Searching expertises of ALL scientists that published as first authors when affiliated to single institution name**

The search will focus on papers and all authors that have published in the past 15 years at a specific institution (academic or otherwise): <br>
The format for a single institution is as follows: **institution="Institution Name"**. <br>
**Caveat**: It is possible that the institutions as input by the user does not match what has been cataloged in ADS, therefore if the final output is empty, make sure to try different versions of the institution names (e.g. Cal Poly Pomona, Cal Poly, California Polytechnic State University) to get the most complete list of authors.

In [11]:
datf=AP.ads_search(institution="Hampton University",refereed="property:refereed",
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
pos(institution:"Hampton University",1)

I am now querying ADS.



In [12]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Afanasev, A.",Hampton University,"Afanasev, A.",2008PhRvL.101l0401A,[Experimental Limit on Optical-Photon Coupling...,2008-09-00,"[12.20.Fv, 11.30.Ly, 12.60.Cn, 13.40.Hq, Exper...","Department of Physics, Hampton University, Ham...",We report on the first results of a sensitive ...,"[10.48550/arXiv.0806.2631, arXiv:0806.2631, 20...","[(light, 4), (coupling, 3), (photon, 3), (resu...","[((scalar, coupling), 2), ((laser, light), 2),...","[((report, results, sensitive), 1), ((results,...",Clean
1,"Ali, Halima","Hampton University, Hampton University","Ali, Halima, Ali, Halima","2011JPlPh..77..785A, 2009JPlPh..75..303A",[Symplectic calculation of the outboard magnet...,"2011-12-00, 2009-06-00",[],"Hampton University, Hampton, VA 23668, USA ( )...",The backward symplectic DIII-D map and continu...,"[10.1017/S0022377811000213, 2011JPlPh..77..785...","[(magnetic, 10), (twm, 10), (perturbation, 9),...","[((field, line), 4), ((flux, preserving), 4), ...","[((flux, preserving, twm), 4), ((field, line, ...","Dirty, Dirty"
2,"Bailey, S. M.",Hampton University,"Bailey, S. M.",1999SoPh..186..243B,[Sounding rocket measurements of the solar sof...,1999-05-00,"[Silicon, Thin Film, Active Area, Reasonable A...","Center for Atmospheric Science, Hampton Univer...",Measurements of the solar soft X-ray (XUV: 2 n...,"[10.1023/A:1005141503558, 1999SoPh..186..243B]","[(solar, 4), (spectrum, 3), (measurement, 2), ...","[((solar, spectrum), 3), ((soft, ray), 2), ((m...","[((model, solar, spectra), 2), ((measurements,...",Dirty
3,"Bailey, Scott M.",Hampton University,"Bailey, Scott M.",1999GeoRL..26.1255B,[Measurements of the solar soft X-ray irradian...,1999-05-00,[Atmospheric Composition and Structure: Thermo...,"Center for Atmospheric Sciences, Hampton Unive...","Beginning on March 11, 1998, the Student Nitri...","[10.1029/1999GL900236, 1999GeoRL..26.1255B]","[(solar, 6), (ray, 3), (irradiances, 3), (cm, ...","[((soft, ray), 2), ((ray, irradiance), 2), ((s...","[((soft, ray, irradiance), 2), ((beginning, ma...",Dirty
4,"Baraka, Suleiman M.",Hampton University,"Baraka, Suleiman M.",2021JGRA..12629528B,[The Impact of Radial and Non-Radial IMF on th...,2021-10-00,"[particle-in-cell, solar wind magnetosphere co...","National Institute of Aerospace, Hampton, VA U...",The boundary between the solar wind and the Ea...,"[2021JGRA..12629528B, 10.1002/essoar.10506965....","[(radial, 6), (mp, 5), (imf, 5), (backstreamin...","[((radial, imf), 4), ((radial, radial), 2), ((...","[((radial, radial, imf), 2), ((boundary, solar...",Dirty
5,"Basu Sarkar, Debajyoti",Hampton University,"Basu Sarkar, Debajyoti",2022GeoRL..4900987B,[Influence of Planetary Surface Temperature on...,2022-11-00,[],Department of Atmospheric and Planetary Scienc...,Heat transport from the interior to the surfac...,"[10.1029/2022GL100987, 2022GeoRL..4900987B]","[(heat, 6), (surface, 4), (terrestrial, 4), (p...","[((heat, pipe), 4), ((heat, transport), 2), ((...","[((heat, pipe, volcanism), 2), ((heat, transpo...",Dirty
6,"Bell, Jared M.",Hampton University,"Bell, Jared M.",2014JGRA..119.4957B,[Developing a self-consistent description of T...,2014-06-00,"[Titan, thermosphere, atmospheric escape, Cass...",Center for Planetary Atmospheres and Flight Sc...,"In this study, we develop a best fit descripti...","[10.1002/2014JA019781, 2014JGRA..119.4957B]","[(escape, 5), (titan, 4), (atmospheric, 4), (u...","[((titan, upper), 3), ((upper, atmosphere), 3)...","[((titan, upper, atmosphere), 3), ((study, dev...",Dirty
7,"Blalock, John J. J.",Hampton University,"Blalock, John J. J.",2019PhDT.......207B,[Measuring seasonal evolution of dynamics and ...,2019-00-00,[],"Hampton University, United States",,[2019PhDT.......207B],[],[],[],Dirty
8,"Box, M. A.","Hampton University, Hampton University","Box, M. A., Box, M. A.","1981JAtS...38.103

# **Example 3: Searching a single author publication while affiliated to a specific institution**

The search will focus on papers published by a single author while they are affiliated to a specific institution, in the past 15 years:<br>

The format for a single author and institution is as follows: **name= 'Last, First', institution= 'Institution Name'**.

In [13]:
datf=AP.ads_search(name= 'Capper, Daniel', institution="University of Southern Mississippi",
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
author:"^Capper, Daniel"pos(institution:"University of Southern Mississippi",1)

I am now querying ADS.



In [14]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Capper, Daniel","University of Southern Mississippi, University...","Capper, Daniel, Capper, Daniel","2022SpPol..5901462C, 2019SpPol..4901325C",[What Should We Do with Our Moon?: Ethics and ...,"2022-02-00, 2019-08-00","[Environmental justice, Environmental preserva...","School of Humanities, University of Southern M...",The planned expansion of the human presence on...,"[10.1016/j.spacepol.2021.101462, 2022SpPol..59...","[(environmental, 8), (human, 7), (moon, 7), (r...","[((baseline, ecology), 3), ((environmental, sc...","[((baseline, ecology, method), 2), ((planned, ...","Dirty, Dirty"


# **Example 4: Searching a single author name within a different time-frame**

The search will focus on papers from one single author that were published in a different time-frame. There are two options for doing so:
   - A single year (e.g. 2010): in this case the code will query ADS for articles published by the specified authors between one year prior to 4  years after. So searching year='2010' will search articles between 2009 and 2014<br>
   - A year range: in this case the syntax is year='[YEAR TO YEAR]' (e.g. year='[2009 TO 2023]') <br>

The format for a single author name remains the same as before: **name= 'Last, First'**. <br>

Here are two examples:
- Searching for Dr. Pepper's articles between year 1999 and 2004
- Searching for Dr. Pepper's articles between year 2019 and 2023

In [15]:
datf=AP.ads_search(name= 'Pepper, Joshua', year='2000',
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
author:"^Pepper, Joshua", pubdate:[1999 TO 2004]

I am now querying ADS.



In [16]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Pepper, Joshua","None, None, None, None, None","Pepper, J., Pepper, Joshua, Pepper, J., Pepper...","2004AAS...20517102P, 2004AIPC..713..185P, 2003...","[Deployment and Initial Data from KELT, KELT: ...","2004-12-00, 2004-06-00, 2003-09-00, 2002-12-00...","[97.82.Fs, 97.82.Cp, Substellar companions, pl...","OSU, Department of Astronomy, 4055 McPherson L...",The Kilodegree Extremely Little Telescope (KEL...,"[2004AAS...20517102P, 2005BAAS...37..375P, 200...","[(star, 15), (transit, 13), (telescope, 12), (...","[((bright, star), 9), ((transit, bright), 8), ...","[((transits, bright, stars), 8), ((bright, sta...","Clean, Dirty, Dirty, Clean, Dirty"


In [17]:
datf=AP.ads_search(name= 'Pepper, Joshua', year='[2019 TO 2023]',
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
author:"^Pepper, Joshua", pubdate:[2019 TO 2023]

I am now querying ADS.



In [18]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Pepper, Joshua","None, None, None, None, None, None, None","Pepper, Joshua, Pepper, Joshua, Pepper, Joshua...","2021plat.confE..96P, 2021tsc2.confE.139P, 2020...","[The TESS Input Catalog and lessons for PLATO,...","2021-10-00, 2021-07-00, 2020-06-00, 2020-01-00...","[Input catalogue, TESS, Zenodo community plato...","Lehigh University, Lehigh University, Departme...",The presentation compares the input catalogues...,"[2021plat.confE..96P, 10.5281/zenodo.5577992, ...","[(star, 14), (tess, 12), (publication, 10), (t...","[((host, star), 5), ((publication, record), 3)...","[((presentation, compares, input), 1), ((compa...","Dirty, Dirty, Clean, Clean, Clean, Clean, Clean"


# **Example 5: Searching a single institution name within a specific time-frame**

The search will focus on authors that publishes as first authors affiliated to a specific institution in a defined timespan. <br>
The format for a author name is the same in previous example (**"Last name, First name"**) and specified year range is similar to the option provided earlier:<br>
   - A single year (e.g. 2010): in this case the code will query ADS for articles published by the specified authors between one year prior to 4  years after. So searching year='2010' will search articles between 2009 and 2014<br>
   - A year range: in this case the syntax is year='[YEAR TO YEAR]' (e.g. year='[2009 TO 2023]') <br>

Following we present two examples:

In [19]:
datf=AP.ads_search(institution="University of Southern Mississippi",year='2000',
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
pos(institution:"University of Southern Mississippi",1), pubdate:[1999 TO 2004]

I am now querying ADS.



In [20]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Buchanan, Randy K.",University of Southern Mississippi,"Buchanan, Randy K.",2003AIPC..684.1121B,[Simulating Martian Temperatures],2003-09-00,"[07.20.Mc, 95.55.-n, Cryogenics, refrigerators...","Electronic Engineering Technology, University ...",The Mars Electrostatic Chamber (MEC) was desig...,"[2003AIPC..684.1121B, 10.1063/1.1627280]","[(temperature, 9), (control, 5), (mar, 4), (ch...","[((temperature, control), 3), ((pressure, atmo...","[((mars, electrostatic, chamber), 1), ((electr...",Dirty


# **Example 6: Searching a single Author, at a specific institution and within a specific time-frame**

The following example combines several of the previous ones in a single search.
Specifically:<br>
   - A single author<br>
   - Affiliated to a single institutions<br>
   - In a specific time frame of publications<br>
    
Please, refer to the previous examples for the sintax required. <br>
Here are an example

In [21]:
datf=AP.ads_search(name= 'Brown, Beth A.', institution="Howard university",year='[2009 TO 2022]',
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
author:"^Brown, Beth A."pos(institution:"Howard university",1), pubdate:[2009 TO 2022]

I am now querying ADS.



In [22]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Brown, Beth A.",Howard university,"Brown, Beth A.",2010AIPC.1280....3B,[Causal Structures of Dynamic Black Holes],2010-10-00,"[black holes, space-time configurations, satel...","NASA Goddard Space Flight Center, Greenbelt, M...","Dynamic space-times, especially those manifest...","[10.1063/1.3507200, arXiv:0904.4192, 2010AIPC....","[(black, 3), (hole, 3), (dynamic, 2), (behavio...","[((black, hole), 3), ((hole, presented), 2), (...","[((black, hole, presented), 2), ((dynamic, spa...",Dirty


# **Example 7: Searching through a list of institutions**

The search will focus on papers from a list of institutions, so the input is a csv file that has multiple institution names stored in it. This will then find all papers from those institutions (**see CAVEATS in Example 2 above related to Institution Names)**:<br>

The input file has to be a .csv file (e.g."top10inst.csv"), and must contain at least one column titled  **"Current Institution"** or **"Institution"** (the first cell of the column is usually interpreted as such). The file can contains other columns, they will be ignored.<br>
If the file is in a different directory than the one where the code it, include the whole path. <br>

The code will run as in Example 2 above for each institutions and append the results at each iteration providing a final dataframe with all the researchers at all the institutions in the list provided.<br>
**NOTE: at the moment if an institution query returns an empty dataframe the code will ignore it and continue to the following one.**


In [23]:
datf=AP.run_file_search(filename='Fellows_Example.csv',
               token=token, stop_dir=stop_dir)



What type of search do you want to conduct?
-Enter 'name' for Name Search - search by author name
-Enter 'institution' for Institution Search - search by institution
-Enter 'fellow' for Fellow Search - search by name, institution, and year
You are running 'name' search.

Listed are the available columns from your dataset: Name, Institution, Fellowship Year
Searching for results...
I will search for papers matching the following criteria:
pos(author:"^Browning, Matthew",1) OR pos(author:"Browning, Matthew",2), pubdate:[2003 TO 2030]

I am now querying ADS.

Completed 1 searches - Processed name: Browning, Matthew (including second author)
I will search for papers matching the following criteria:
pos(author:"^Cruz, Kelle",1) OR pos(author:"Cruz, Kelle",2), pubdate:[2003 TO 2030]

I am now querying ADS.

Ooops, something went wrong.

Completed 2 searches - Processed name: Cruz, Kelle (including second author)
I will search for papers matching the following criteria:
pos(author:"^Gawiser,

In [ ]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Browning, Matthew","None, None, None, None, None, None, None, None...","Käpylä, Petri J., Joshi-Cale, Tom, Browning, M...","2023SSRv..219...58K, 2022csss.confE.210J, 2021...",[Simulations of Solar and Stellar Dynamos and ...,"2023-10-00, 2022-12-00, 2021-03-00, 2020-01-00...","[Dynamo, Magnetohydrodynamics, Simulation, Tur...","Institute for Astrophysics and Geophysics, Uni...",We review the state of the art of three dimens...,"[2023arXiv230516790K, 10.1007/s11214-023-01005...","[(field, 207), (magnetic, 172), (convection, 1...","[((magnetic, field), 111), ((differential, rot...","[((anelastic, spherical, harmonic), 20), ((sph...",Dirty
1,"Cruz, Kelle","None, None, None, None, None, None, None, None...","Rodriguez, D. R., Cruz, Kelle, Baldelli, Alexa...","2024ASPC..535...33R, 2024ESS.....562615C, 2024...",[The SIMPLE Archive: Building a Collaborative ...,"2024-05-00, 2024-04-00, 2024-02-00, 2024-02-00...","[Very low mass stars, Astronomy, archives, Ast...","-, CUNY Hunter College &amp; AMNH, American Mu...",We present the SIMPLE Archive alongside a Pyth...,"[2024ASPC..535...33R, 2024adass..31...33R, 202...","[(dwarf, 302), (brown, 110), (spectral, 109), ...","[((brown, dwarf), 110), ((spectral, type), 45)...","[((low, mass, stars), 18), ((stars, brown, dwa...",Dirty
2,"Gawiser, Eric","None, None, None, None, None, None, None, None...","Firestone, Nicole M., Firestone, Nicole M., Kh...","2025arXiv250108568F, 2024ApJ...974..217F, 2024...",[ODIN: Star Formation Histories Reveal Formati...,"2025-01-00, 2024-10-00, 2024-09-00, 2024-09-00...","[Astrophysics - Astrophysics of Galaxies, Emis...","-, Department of Physics and Astronomy, Rutger...","In this work, we test the conventional assumpt...","[arXiv:2501.08568, 10.48550/arXiv.2501.08568, ...","[(galaxy, 562), (star, 216), (redshift, 205), ...","[((star, formation), 169), ((formation, rate),...","[((star, formation, rate), 33), ((star, format...",Dirty


# **Example 8: Searching through a list of Authors names**

The search will focus on papers from a list of authors names (similar format as Example 1 above, **'Last, First'**). <br>
The input is a .csv file that has multiple authors names stored in it under a column Title: **"Name"**. <br>
The ADS search will focus on the period 2003 to 2023.
<br>
If the file is in a different directory than the one where the code it, include the whole path. <br>

The code will then execute the search one name after the other and uppend each result to the previous one.<br>
In the following example we use, for convenience, the same example file as before which also contain a list of researchers names.


In [ ]:
datf=AP.run_file_search(filename='Fellows_Example.csv',
               token=token, stop_dir=stop_dir)



What type of search do you want to conduct?
-Enter 'name' for Name Search - search by author name
-Enter 'institution' for Institution Search - search by institution
-Enter 'fellow' for Fellow Search - search by name, institution, and year
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or 'fellow'.
Invalid search type. Please enter 'name', 'institution', or

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1062f1040>>
Traceback (most recent call last):
  File "/Users/sayemkamal/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Completed 1 searches - Processed name: Browning, Matthew (including second author)
I will search for papers matching the following criteria:
pos(author:"^Cruz, Kelle",1) OR pos(author:"Cruz, Kelle",2), pubdate:[2003 TO 2030]

I am now querying ADS.

Completed 2 searches - Processed name: Cruz, Kelle (including second author)
I will search for papers matching the following criteria:
pos(author:"^Gawiser, Eric",1) OR pos(author:"Gawiser, Eric",2), pubdate:[2003 TO 2030]

I am now querying ADS.



In [ ]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

In [ ]:
print(len(datf.iloc[0]['Title']))

# Example 9: Searching a list of Authors at Institutionsd during specific times

In [ ]:
datf=AP.run_file_search(filename='Fellows_Example.csv',
               token=token, stop_dir=stop_dir)

In [ ]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")